**Black Friday Sales using AutoML**

**About Data Set**

A retail company wants to comprehend and interpret the purchase behavior of their customers, with respect to various products belonging to different categories, for the Black Friday sale. The data available for the company is the purchase summary of customers which includes customer demographics, product information, the total purchase amount, for chosen high volume products sold during the Black Friday period of last year.

The objective is developing a model to make predictions on the purchasing capacity of their customers with respect to various products, aiding them in creating personalized offers for customers against different products along with understanding which areas need to increase sales during Black Friday

In [45]:
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u342-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [46]:
!java -version

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode)


In [47]:
#Installing H2O library
!pip install H2O

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
#Importing Libraries
%matplotlib inline
import random, os, sys
import h2o
import pandas
import pprint
import operator
import matplotlib
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from tabulate import tabulate
from h2o.automl import H2OAutoML
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np

In [49]:
#Initialising H2O instance
h2o.init(strict_version_check=False)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 17 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_x4n48s
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.313 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [50]:
#Loading dataset 
dfimport = h2o.import_file(path = "/content/train.csv")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


**MultiColinearity**

In [ ]:
df_importpd = pd.read_csv("/content/train.csv")

#plot color scaled correlation matrix
corr=df_importpd.corr()
corr.style.background_gradient(cmap='coolwarm')

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
User_ID,1.000000,-0.023971,0.020443,0.003825,0.001529,0.003419,0.004716
Occupation,-0.023971,1.000000,0.024280,-0.007618,-0.000384,0.013263,0.020833
Marital_Status,0.020443,0.024280,1.000000,0.019888,0.015138,0.019473,-0.000463
Product_Category_1,0.003825,-0.007618,0.019888,1.000000,0.540583,0.229678,-0.343703
Product_Category_2,0.001529,-0.000384,0.015138,0.540583,1.000000,0.543649,-0.209918
Product_Category_3,0.003419,0.013263,0.019473,0.229678,0.543649,1.000000,-0.022006
Purchase,0.004716,0.020833,-0.000463,-0.343703,-0.209918,-0.022006,1.000000


Product Category 2 & product Category 3 are highly correlated.

Product Category 1 & product Category 3 also show some level of correlation.


**Data Cleaning**

In [51]:
#Removing rows with null values 
newframe=dfimport.na_omit()
df = newframe
df.describe()

Rows:141459
Cols:12

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
type,int,enum,enum,enum,int,enum,int,int,int,int,int,int
mins,1000001.0,,,,0.0,,0.0,0.0,1.0,2.0,3.0,185.0
mean,1003050.313334605,,,,8.155819000558466,,1.4783152715627863,0.40448469167744716,2.7376130186131653,6.885804367343183,12.65886228518509,11651.448405545067
maxs,1006040.0,,,,20.0,,3.0,1.0,15.0,16.0,18.0,23959.0
sigma,1730.583654879296,,,,6.484714073640686,,0.9898901933053615,0.49079377409580127,2.5672754395767443,4.49440458564803,4.129737896841566,5085.821272724652
zeros,0,,,,17801,,22389,84241,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,1000001.0,P00248942,F,0-17,10.0,A,2.0,0.0,1.0,6.0,14.0,15200.0
1,1000004.0,P00184942,M,46-50,7.0,B,2.0,1.0,1.0,8.0,17.0,19215.0
2,1000005.0,P00145042,M,26-35,20.0,A,1.0,1.0,1.0,2.0,5.0,15665.0


**Binary Classification**

In [52]:
#Splitting up and storing dependent and independent variable
dfcopy = df
myY = "Gender"
myX = ["User_ID", "Product_ID", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [53]:
#Splitting up the data set into training, testing and validation sets
df_train,df_test,df_valid = dfcopy.split_frame(ratios=[.8, .10])

print ("Rows in Train",df_train.nrow)
print ("Rows in Validation",df_valid.nrow)
print ("Rows in Test",df_test.nrow)

Rows in Train 112998
Rows in Validation 14220
Rows in Test 14241


In [54]:
ml = H2OAutoML(max_models = 10, seed = 10, verbosity = "info", nfolds = 0)

**seed**: Set a seed for reproducibility. AutoML guarantees reproducibility only under certain conditions. By default, for performance reasons, H2O Deep Learning models are not reproducible, so if the user requires reproducibility, then exclude_algos must contain "DeepLearning". In addition max_models must be used because max_runtime_secs is resource limited, meaning that if the available compute resources are not the same between runs, AutoML may be able to train more models on one run vs another. Defaults to NULL/None.

**verbosity**: This field is optional and defaults to NULL/None. This is the verbosity of the backend messages printed during training.We can mention "debug", "info", "warn".

**nfolds**: This is an optional field. A value of greater than or equal to 2 is to be specifieda for the number of folds for k-fold cross-validation of the models in the AutoML run. A value of “-1” can be specified in order to let AutoML choose if k-fold cross-validation or blending mode should be used. To disable cross-validation we can specify 0.

In [55]:
#Training the model
ml.train(x = myX, y = myY, training_frame = df_train, validation_frame = df_valid)

AutoML progress: |
03:41:36.335: Project: AutoML_4_20221108_34136
03:41:36.335: Cross-validation disabled by user: no fold column nor nfolds > 1.
03:41:36.335: Setting stopping tolerance adaptively based on the training frame: 0.00297484691273901
03:41:36.335: Build control seed: 10
03:41:36.335: training frame: Frame key: AutoML_4_20221108_34136_training_py_35_sid_8b5d    cols: 12    rows: 112998  chunks: 8    size: 1813864  checksum: -3389356373312444664
03:41:36.335: validation frame: Frame key: py_37_sid_8b5d    cols: 12    rows: 14220  chunks: 8    size: 505054  checksum: -3391914661864499368
03:41:36.335: leaderboard frame: Frame key: py_37_sid_8b5d    cols: 12    rows: 14220  chunks: 8    size: 505054  checksum: -3391914661864499368
03:41:36.335: blending frame: NULL
03:41:36.335: response column: Gender
03:41:36.335: fold column: null
03:41:36.335: weights column: null
03:41:36.340: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_4_20221108_34136


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    275                275                         1.38435e+06            0            10           7.30909       1             812           318.524

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.005273554957484822
RMSE: 0.07261924646734377
LogLoss: 0.04114732486618086
Mean Per-Class Error: 0.0029632768871644443
AUC: 0.999962180721734
AUCPR: 0.999988589597632
Gini: 0.9999243614434681

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6166050140027257
       F      M      Error    Rate
-----  -----  -----  -------  ----------------
F      25750  132    0.0051   (132.0/25882.0)
M      72     87044  0.0008   (72.0/87116.0)
Total  25822  87176  0.0018   (204.0/112998.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.616605     0.99883   188
max f2                       0.584724     0.999142  192
max f0point5                 0.701896     0.99903   176
max accuracy                 0.616605     0.998195  188
max precision                0.999266     1         0
max recall                   0.230577     1         265
max specificity              0.999266     1         0
max absolute_mcc             0.616605     0.994885  188
max min_per_class_accuracy   0.71482      0.997716  174
max mean_per_class_accuracy  0.701896     0.997807  176
max tns                      0.999266     25882     0
max fns                      0.999266     86390     0
max fps                      0.00221999   25882     399
max tps                      0.230577     87116     265
max tnr                      0.999266     1         0
max fnr                      0.999266     0.991666  0
max fpr                      0.00221999   1         399
max tpr                      0.230577     1         265

Gains/Lift Table: Avg response rate: 77.10 %, avg score: 77.10 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100002                   0.998836           1.2971    1.2971             1                0.999167   1                           0.999167            0.0129712       0.0129712                  29.7098   29.7098            0.0129712
2        0.0200004                   0.998377           1.2971    1.2971             1                0.998598   1                           0.998883            0.0129712       0.0259424                  29.7098   29.7098            0.0259424
3        0.0300005                   0.997981           1.2971    1.2971             1                0.998177   1                           0.998648            0.0129712       0.0389136                  29.7098   29.7098            0.0389136
4        0.0400007                   0.997647           1.2971    1.2971             1                0.997812   1                           0.998439            0.0129712       0.0518848                  29.7098   29.7098            0.0518848
5        0.0500009                   0.997344           1.2971    1.2971             1                0.997495   1              

MSE: 0.005273554957484822

RMSE: 0.07261924646734377

LogLoss: 0.04114732486618086

Mean Per-Class Error: 0.0029632768871644443

The predicted values, for binary classification, make sense.

In [81]:
lb = ml.leaderboard
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_6_20221108_42128,3475.86,1.20816e+07,2695.41,nan,1.20816e+07
GBM_1_AutoML_6_20221108_42128,3487.63,1.21636e+07,2707.06,nan,1.21636e+07
GBM_3_AutoML_6_20221108_42128,3489.34,1.21755e+07,2716.55,0.365963,1.21755e+07
XGBoost_3_AutoML_6_20221108_42128,3505.49,1.22885e+07,2735.2,nan,1.22885e+07
GBM_2_AutoML_6_20221108_42128,3505.54,1.22888e+07,2736.65,0.368117,1.22888e+07
XGBoost_2_AutoML_6_20221108_42128,3516.96,1.2369e+07,2720.46,nan,1.2369e+07
XRT_1_AutoML_6_20221108_42128,3541.29,1.25407e+07,2772.6,0.374721,1.25407e+07
XGBoost_1_AutoML_6_20221108_42128,3549.21,1.25969e+07,2734.34,nan,1.25969e+07
DRF_1_AutoML_6_20221108_42128,3651.08,1.33304e+07,2803.67,0.374729,1.33304e+07
GLM_1_AutoML_6_20221108_42128,5082.06,2.58273e+07,4217.76,0.609805,2.58273e+07


Leaderboard - The AutoML object includes a “leaderboard” of models that were trained in the process. The number of folds used in the model evaluation process can be adjusted using the nfolds parameter.

In [57]:
# finding and storing the best model and rinting the output
best_model = h2o.get_model(ml.leaderboard[1,'model_id'])
best_model.algo

'xgboost'

For tree-based models like Gradient Boosting there are no model assumptions to validate.

Learning rate and n_estimators are two critical hyperparameters for gradient boosting decision trees

In [80]:
df_pred=ml.leader.predict(df_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [59]:
df_pred.head()

predict,F,M
M,0.011199,0.988801
F,0.987265,0.0127355
F,0.987616,0.012384
M,0.00509308,0.994907
M,0.00816299,0.991837
M,0.00706539,0.992935
M,0.0171497,0.98285
F,0.97118,0.0288201
M,0.00129117,0.998709
F,0.969978,0.030022


In [60]:
ml.leader.model_performance(df_test)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.010814313012107682
RMSE: 0.1039918891650098
LogLoss: 0.058736218518827196
Mean Per-Class Error: 0.011862188541485484
AUC: 0.9990533123944978
AUCPR: 0.9996661923273066
Gini: 0.9981066247889956

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6371536525773253
       F     M      Error    Rate
-----  ----  -----  -------  --------------
F      3219  69     0.021    (69.0/3288.0)
M      30    10923  0.0027   (30.0/10953.0)
Total  3249  10992  0.007    (99.0/14241.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.637154     0.995489  181
max f2                       0.55074      0.997083  196
max f0point5                 0.74715      0.995493  157
max accuracy                 0.637154     0.993048  181
max precision                0.999116     1         0
max recall                   0.208548     1         276
max specificity              0.999116     1         0
max absolute_mcc             0.637154     0.980374  181
max min_per_class_accuracy   0.783074     0.988588  149
max mean_per_class_accuracy  0.74715      0.989991  157
max tns                      0.999116     3288      0
max fns                      0.999116     10822     0
max fps                      0.0021576    3288      399
max tps                      0.208548     10953     276
max tnr                      0.999116     1         0
max fnr                      0.999116     0.98804   0
max fpr                      0.0021576    1         399
max tpr                      0.208548     1         276

Gains/Lift Table: Avg response rate: 76.91 %, avg score: 77.42 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100414                   0.998762           1.30019     1.30019            1                0.999106   1                           0.999106            0.0130558       0.0130558                  30.0192   30.0192            0.0130558
2        0.0200126                   0.998326           1.30019     1.30019            1                0.998548   1                           0.998828            0.0129645       0.0260203                  30.0192   30.0192            0.0260203
3        0.0300541                   0.997918           1.30019     1.30019            1                0.99811    1                           0.998588            0.0130558       0.0390761                  30.0192   30.0192            0.0390761
4        0.0400253                   0.997523           1.30019     1.30019            1                0.997712   1                           0.99837             0.0129645       0.0520405                  30.0192   30.0192            0.0520405
5        0.0500667                   0.99727            1.30019     1.30019            1                0.997392   1                           0.998174            0.0130558       0.0650963                  30.0192   30.0192            0.0650963
6        0.100063                    0.995909           1.30019     1.30019            1                0.996582   1                           0.997379            0.065005        0.130101                   30.0192   30.0192            0.130101
7        0.15006                     0.994488           1.30019     1.30019            1                0.995176   1                           0.996645            0.065005        0.195106                   30.0192   30.0192            0.195106

In [61]:
# Multiclass Classification
dfcopy_multi = df

myY = "City_Category"
myX = ["Age", "Occupation", "Gender", "Stay_In_Current_City_Years", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [62]:
#Splitting data into training, test and validation sets
df_train,df_test,df_valid = dfcopy_multi.split_frame(ratios=[.8, .10])

print ("Rows in Train",df_train.nrow)
print ("Rows in Validation",df_valid.nrow)
print ("Rows in Test",df_test.nrow)

Rows in Train 113217
Rows in Validation 14131
Rows in Test 14111


In [63]:
ml = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0)

In [64]:
ml.train(x = myX, y = myY, training_frame = df_train, validation_frame = df_valid)

AutoML progress: |
04:16:03.463: Project: AutoML_5_20221108_41603
04:16:03.463: Cross-validation disabled by user: no fold column nor nfolds > 1.
04:16:03.463: Setting stopping tolerance adaptively based on the training frame: 0.0029719683395997114
04:16:03.463: Build control seed: 10
04:16:03.463: training frame: Frame key: AutoML_5_20221108_41603_training_py_44_sid_8b5d    cols: 12    rows: 113217  chunks: 8    size: 1816765  checksum: -3356065052680920681
04:16:03.464: validation frame: Frame key: py_46_sid_8b5d    cols: 12    rows: 14131  chunks: 8    size: 503875  checksum: -3358272111842371636
04:16:03.464: leaderboard frame: Frame key: py_46_sid_8b5d    cols: 12    rows: 14131  chunks: 8    size: 503875  checksum: -3358272111842371636
04:16:03.464: blending frame: NULL
04:16:03.464: response column: City_Category
04:16:03.464: fold column: null
04:16:03.464: weights column: null
04:16:03.464: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w)

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_5_20221108_41603


Model Summary: 
    number_of_trees
--  -----------------
    55

ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.2420687844984086
RMSE: 0.49200486227110457
LogLoss: 0.6743265510853835
Mean Per-Class Error: 0.3194361994553368
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A      B      C      Error     Rate
-----  -----  -----  --------  ----------------
17580  6932   3772   0.378447  10,704 / 28,284
4783   36662  5775   0.223592  10,558 / 47,220
4009   9427   24277  0.35627   13,436 / 37,713
26372  53021  33824  0.306473  34,698 / 113,217

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.693527
2    0.923987
3    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.2856793007900962
RMSE: 0.5344897574229989
LogLoss: 0.787811359100479
Mean Per-Class Error: 0.39621794142026284
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A     B     C     Error     Rate
----  ----  ----  --------  --------------
1873  1079  572   0.468502  1,651 / 3,524
768   4106  1005  0.301582  1,773 / 5,879
570   1409  2749  0.41857   1,979 / 4,728
3211  6594  4326  0.382351  5,403 / 14,131

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.617649
2    0.890807
3    1

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_logloss    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_classification_error    validation_auc    validation_pr_auc
--  -------------------  ----------  -----------------  ---------------  ------------------  -------------------------------  --------------  -----------------  -----------------  --------------------  ---------------------------------  ----------------  -------------------
    2022-11-08 04:16:03  0.007 sec   0                  0.666667         1.09861             0.666896                         nan             nan                0.666667           1.09861               0.665416                           nan               nan
    2022-11-08 04:16:04  1.337 sec   5                  0.583128         0.884733            0.369185                         nan             nan                0.590894           0.905232              0.392612                           nan               nan
    2022-11-08 04:16:06  2.705 sec   10                 0.546232         0.799879            0.351617                         nan             nan                0.559718           0.834874              0.38412                            nan               nan
    2022-11-08 04:16:07  4.334 sec   15                 0.527231         0.756483            0.338695                         nan             nan                0.546364           0.806104              0.379379                           nan               nan
    2022-11-08 04:16:09  6.095 sec   20                 0.518637         0.7366              0.331072                         nan             nan                0.54199            0.797503              0.381572                           nan               nan
    2022-11-08 04:16:11  8.195 sec   25                 0.511706         0.72039        


MSE: 0.2420687844984086

RMSE: 0.49200486227110457

LogLoss: 0.6743265510853835

Mean Per-Class Error: 0.3194361994553368

The predicted values make sense for multiclass classification.
So the model makes sense.

In [65]:
lb = ml.leaderboard
lb.head()

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_5_20221108_41603,0.396218,0.787811,0.53449,0.285679
XGBoost_2_AutoML_5_20221108_41603,0.397843,0.78861,0.536566,0.287903
GBM_4_AutoML_5_20221108_41603,0.399301,0.811989,0.549663,0.302129
GBM_1_AutoML_5_20221108_41603,0.400902,0.822291,0.552369,0.305111
DRF_1_AutoML_5_20221108_41603,0.414946,0.830992,0.545416,0.297479
XGBoost_3_AutoML_5_20221108_41603,0.429686,0.85703,0.566744,0.321199
GBM_3_AutoML_5_20221108_41603,0.431502,0.867977,0.573409,0.328798
GBM_2_AutoML_5_20221108_41603,0.453452,0.898587,0.585876,0.34325
XRT_1_AutoML_5_20221108_41603,0.499446,0.950094,0.6073,0.368814
GLM_1_AutoML_5_20221108_41603,0.643759,1.0585,0.647206,0.418876


In [66]:
# finding and storing the best model and printing the output
best_model = h2o.get_model(ml.leaderboard[0,'model_id'])
best_model.algo

'xgboost'

For Tree based models like Gradient Boosting, no model assumptions are present to validate.

Learning rate and n_estimators are two critical hyperparameters for gradient boosting decision trees

In [67]:
df_pred=ml.leader.predict(df_test)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [68]:
df_pred.head()

predict,A,B,C
B,0.281852,0.607102,0.111046
B,0.123895,0.800839,0.0752663
A,0.50238,0.212688,0.284933
C,0.0414721,0.21641,0.742118
C,0.00700037,0.00684467,0.986155
B,0.00696692,0.864182,0.128851
C,0.239891,0.32741,0.432699
B,0.250075,0.500972,0.248953
C,0.0600912,0.0320156,0.907893
A,0.414308,0.413756,0.171936


In [69]:
ml.leader.model_performance(df_test)

ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.28800484427023837
RMSE: 0.5366608279632848
LogLoss: 0.7944731101936067
Mean Per-Class Error: 0.40293859880754895
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A     B     C     Error     Rate
----  ----  ----  --------  --------------
1872  1056  599   0.469237  1,655 / 3,527
771   4145  1006  0.300068  1,777 / 5,922
609   1440  2613  0.439511  2,049 / 4,662
3252  6641  4218  0.38842   5,481 / 14,111

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.61158
2    0.885338
3    1

In [70]:
# Regression 
dfcopy_reg = df

myY = "Purchase"
myX = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [71]:
#Splitting data into training, test and validation sets
df_train,df_test,df_valid = dfcopy_multi.split_frame(ratios=[.8, .10])

print ("Rows in Train",df_train.nrow)
print ("Rows in Validation",df_valid.nrow)
print ("Rows in Test",df_test.nrow)

Rows in Train 113058
Rows in Validation 14102
Rows in Test 14299


In [72]:
ml = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0)

In [73]:
ml.train(x = myX, y = myY, training_frame = df_train, validation_frame = df_valid)

AutoML progress: |
04:21:28.345: Project: AutoML_6_20221108_42128
04:21:28.345: Cross-validation disabled by user: no fold column nor nfolds > 1.
04:21:28.345: Setting stopping tolerance adaptively based on the training frame: 0.0029740574307812262
04:21:28.345: Build control seed: 10
04:21:28.345: training frame: Frame key: AutoML_6_20221108_42128_training_py_53_sid_8b5d    cols: 12    rows: 113058  chunks: 8    size: 1814656  checksum: -3392719601830047457
04:21:28.345: validation frame: Frame key: py_55_sid_8b5d    cols: 12    rows: 14102  chunks: 8    size: 503490  checksum: -3387401540773731442
04:21:28.345: leaderboard frame: Frame key: py_55_sid_8b5d    cols: 12    rows: 14102  chunks: 8    size: 503490  checksum: -3387401540773731442
04:21:28.345: blending frame: NULL
04:21:28.345: response column: Purchase
04:21:28.345: fold column: null
04:21:28.345: weights column: null
04:21:28.345: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), gri

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,80.0,80.0,512185.0,0.0,10.0,8.0,1.0,759.0,505.5625
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2022-11-08 04:22:22,0.003 sec,0.0,5087.5805235,4225.6682842,25883475.5834307,5082.0793496,4217.7758456,25827530.5151325
,2022-11-08 04:22:23,0.414 sec,5.0,4388.3400002,3646.4606367,19257527.9577057,4410.5866510,3666.7080275,19453274.6061133
,2022-11-08 04:22:23,0.813 sec,10.0,3918.1459916,3231.6657340,15351868.0115935,3967.5094129,3276.4296094,15741130.9415072
,2022-11-08 04:22:24,1.242 sec,15.0,3614.1233636,2931.2923790,13061887.6874282,3683.2169022,2991.3943447,13566086.7488545
,2022-11-08 04:22:24,1.661 sec,20.0,3508.9708632,2803.7886270,12312876.5186302,3598.0642842,2879.6917419,12946066.5933064
,2022-11-08 04:22:24,2.085 sec,25.0,3443.1179653,2718.7359879,11855061.3232002,3550.7507209,2809.8147141,12607830.6818794
,2022-11-08 04:22:25,2.530 sec,30.0,3398.4489152,2654.0300316,11549455.0292909,3519.5476449,2758.2808389,12387215.6245089
,2022-11-08 04:22:25,2.960 sec,35.0,3371.5928456,2621.1254571,11367638.3164821,3506.4389864,2736.9745511,12295114.3652268


In [74]:
lb = ml.leaderboard
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_6_20221108_42128,3475.86,1.20816e+07,2695.41,nan,1.20816e+07
GBM_1_AutoML_6_20221108_42128,3487.63,1.21636e+07,2707.06,nan,1.21636e+07
GBM_3_AutoML_6_20221108_42128,3489.34,1.21755e+07,2716.55,0.365963,1.21755e+07
XGBoost_3_AutoML_6_20221108_42128,3505.49,1.22885e+07,2735.2,nan,1.22885e+07
GBM_2_AutoML_6_20221108_42128,3505.54,1.22888e+07,2736.65,0.368117,1.22888e+07
XGBoost_2_AutoML_6_20221108_42128,3516.96,1.2369e+07,2720.46,nan,1.2369e+07
XRT_1_AutoML_6_20221108_42128,3541.29,1.25407e+07,2772.6,0.374721,1.25407e+07
XGBoost_1_AutoML_6_20221108_42128,3549.21,1.25969e+07,2734.34,nan,1.25969e+07
DRF_1_AutoML_6_20221108_42128,3651.08,1.33304e+07,2803.67,0.374729,1.33304e+07
GLM_1_AutoML_6_20221108_42128,5082.06,2.58273e+07,4217.76,0.609805,2.58273e+07


In [79]:
# finding and storing the best model and rinting the output
best_model = h2o.get_model(ml.leaderboard[0,'model_id'])
best_model.algo

'gbm'

A GBM model has two categories of hyperparameters that are boosting hyperparameters and tree-specific hyperparameters. There are two main boosting hyperparameters which include: Number of trees: The total number of trees in the sequence or ensemble.

In [76]:
df_pred=ml.leader.predict(df_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [77]:
df_pred.head()

predict
6724.08
12839.9
13371.2
11906.8
6990.39
12407.9
9312.81
3157.94
14632.1
11350.1


In [78]:
ml.leader.model_performance(df_test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 12156216.000392227
RMSE: 3486.576544461949
MAE: 2694.9909726791357
RMSLE: NaN
Mean Residual Deviance: 12156216.000392227

Since we have not done any pre-processing of the data, we are getting such high MSE and RMSE values. 


**Conclusion**

For the given dataset, Binary and multiclass classification give appropriate predictions.

This dataset has a numeric target variable and therefore it requires feature engineering and appropriate data pre-processing to get better preditions.

**References**

6105_H2O_automl_lending_club.ipynb - https://github.com/aiskunks/Skunks_Skool/blob/main/INFO_6105/6105/6105_Airlines_GBM_AutoML.ipynb

Sckit learn offcial documentation

h2o.ai Documentation

Kaggle

Refered to https://towardsdatascience.com/back-to-basics-assumptions-of-common-machine-learning-models-e43c02325535 article to understand model assumptions

Used h2o.ai for autoML implementation

https://www.kaggle.com/code/gopikrishnamashetty/black-friday-sales-eda-prediction/notebook

https://www.kaggle.com/code/margesh/regression-scikit-xgb-h2o-automl